<a href="https://colab.research.google.com/github/Kinds-of-Intelligence-CFI/comparative-object-permanence/blob/measurement-layout/analysis/measurement-layouts/object_permanence_measurement_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparative Object Permanence Measurement Layouts

Authors: K. Voudouris, J. Burden, J. Hernández-Orallo

## INIT

In [ ]:
!pip install pymc --quiet
!pip install numpy --quiet
!pip install arviz --quiet
!pip install erroranalysis --quiet

In [ ]:
import arviz as az
import erroranalysis as ea
import gc
import graphviz
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import random as rm
import seaborn as sns

from IPython.display import Image
from scipy import stats
from sklearn.model_selection import train_test_split
from google.colab import files

print(f"Running on PyMC v{pm.__version__}")

## Load Data

In [ ]:
agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_final_clean_agents_wide.csv?token=GHSAT0AAAAAACEGARGQY7IJDAXM63HIYWZIZG75C4A'
agent_data = pd.read_csv(agents_url)

children_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_final_clean_children_wide.csv?token=GHSAT0AAAAAACEGARGQXTFZR34LOMM4BZIOZG75DGA'
children_data = pd.read_csv(children_url)

synthetic_agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/measurement-layout/analysis/measurement-layouts/results_synthetic_agents_wide.csv?token=GHSAT0AAAAAACEGARGQWE57GWCYBGAO756CZG75DRA'
synthetic_agents_data = pd.read_csv(synthetic_agents_url)


## Define the Measurement Layout

In [ ]:
## Helper functions

def logistic(x):
  return 1 / (1 + np.exp(-x))

def visualAcuityLOMargin(ability, goalSmallness): # must return a value between -inf and inf  (more precisely between -maxVisualAcuityAbility and maxVisualAcuityAbility)
  return ability - goalSmallness   # Goes between -inf to inf, with logodds=0 meaning this would lead to 0.5 chance of success

def SimplePrMargin(ability, binaryFeature): # must return a value between 0 and 1
  return 1-((1-ability)*binaryFeature)  # If binaryFeature is 0 then the margin represents p(success)=1. If binaryFeature = 1 then p(success)=ability

#def flatNavMargin(ability, distanceToGoal, numTurns, allocentricOcclusion): # must return a value between -inf and inf  (more precisely between -maxSpatialAbility and maxSpatialAbility)
def flatNavMargin(ability, distanceToGoal, numTurns):
  #return ability - ((distanceToGoal * numTurns)*allocentricOcclusion)   # Goes between -inf to inf, with logodds=0 meaning this would lead to 0.5 chance of success
  return ability - ((distanceToGoal * numTurns))

def objPermanenceLOMargin(ability, allocentricOcclusion, cvChickP, pctbP, lightsOut, distanceToGoal, numPositions, maxPermAbility, lightsOutPenalisation, uniformAbilitySlack): #,  visualAcuityP): # must return a value between -inf and inf (more precisely between -maxPermAbility and maxPermAbility)
#def objPermanenceLOMargin(ability, allocentricOcclusion, lightsOut, distanceToGoal, numPositions, maxPermAbility, lightsOutPenalisation, uniformAbilitySlack):
  assert lightsOutPenalisation < uniformAbilitySlack and lightsOutPenalisation > 1, "Penalisation for lights out must be lower than the slack on abilities."
  OPPerformance = ability  - ((distanceToGoal * numPositions)*(lightsOut*lightsOutPenalisation)) #this returns a value that is lower when (a) ability is lower, (b) the goal is occluded for longer or there are more positions where it could be occluded, or (c) when the lights go out (by a small penalisation)
  ability = maxPermAbility - ((maxPermAbility-OPPerformance)*allocentricOcclusion) * cvChickP * pctbP # this returns an ability value that is modulated by performance on the different paradigms. The agent needs to be good at both to have a high OP ability
  #return maxPermAbility-((maxPermAbility-OPPerformance)*((allocentricOcclusion*cvChick) + (allocentricOcclusion*pctb))) #multiply by a factor that introduces whether the instance is a an OP CV task or an OP PCTB task (independent, so ((allocentricOcclusion*cvChick) + (allocentricOcclusion*pctb)) should be 0 or 1)
  #return maxPermAbility - ((maxPermAbility-OPPerformance)*allocentricOcclusion)
  return ability


In [ ]:
def SetupModel(taskResultsAll, uniformAbilitySlack, agent_type, agent_name = None, sample = 500):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests.
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  if agent_type == "agent":
    taskResults = taskResultsAll.dropna(subset = [agent_name])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults[agent_name]
  elif agent_type == "child":
    taskResults = taskResultsAll.dropna(subset = ['success'])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults['success']
  else:
    print("Agent not recognised. Quitting.")
    return



  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistance = taskResults["cityBlockDistanceToGoal"].max()
  maxTurns = taskResults["minNumTurnsRequired"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max()
  maxPermAbility = maxDistance * maxChoices * uniformAbilitySlack
  #maxMemoryAbility = maxDistance * uniformAbilitySlack
  m = pm.Model()
  with m:

    objPermAbility = pm.Uniform("objPermAbility", 0, maxPermAbility)  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude
    abilityMin["objPermAbility"] = 0
    abilityMax["objPermAbility"] = maxPermAbility

    #memoryAbility = pm.Uniform("memoryAbility", 0, maxMemoryAbility)  # [0,maxMemoryAbility] This is the same as above, but we have the ability in the right magnitude
    #abilityMin["memoryAbility"] = 0
    #abilityMax["memoryAbility"] = maxMemoryAbility

    #Visual acuity
    maxVisualAcuityAbility = maxGoalSize* uniformAbilitySlack
    visualAcuityAbility = pm.Uniform("visualAcuityAbility", 0, maxVisualAcuityAbility)
    abilityMin["visualAcuityAbility"] = 0
    abilityMax["visualAcuityAbility"] = maxVisualAcuityAbility

    # Flat Navigation Ability
    flatNavAbility = pm.Uniform("flatNavAbility", 0, maxDistance*maxTurns)      # how much navigation is involved, i.e, how far away and how circuitous is the path to the goal?
    abilityMin["flatNavAbility"] =0
    abilityMax["flatNavAbility"] = maxDistance*maxTurns

    # Lava Ability
    lavaAbility = pm.Beta("lavaAbility", 1,1)                        # [0,1] Specific ability with lava
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 1

    # Ramp Ability
    rampAbility = pm.Beta("rampAbility",1,1)                          # [0,1] Specific ability with ramps
    abilityMin["rampAbility"] = 0
    abilityMax["rampAbility"] = 1

    # Goal Right Ability
    rightAbility = pm.Beta("rightAbility", 1, 1)
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 1

    # Goal Left Ability
    leftAbility = pm.Beta("leftAbility", 1, 1)
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 1

    # Goal Ahead Ability
    aheadAbility = pm.Beta("aheadAbility", 1, 1)
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 1

    # CV Chick Ability
    CVChickAbility = pm.Beta("CVChickAbility", 1, 1)
    abilityMin["CVChickAbility"] = 0
    abilityMax["CVChickAbility"] = 1

    # PCTB Ability
    PCTBAbility = pm.Beta("PCTBAbility", 1, 1)
    abilityMin["PCTBAbility"] = 0
    abilityMax["PCTBAbility"] = 1



    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values)
    rampPresence = pm.MutableData("rampPresence", taskResults["taskCriticalRampPresence"].values)
    lightsOutPresence = pm.MutableData("lightsOutPresence", taskResults["lightsOutPresence"].values)
    #numGoals = pm.MutableData("numberOfGoals", taskResults["numGoalsAll"].values)
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values)
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values)
    goalDist = pm.MutableData("goalDistance", taskResults["cityBlockDistanceToGoal"])
    numTurns = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsRequired"])
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"])
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"])
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"])
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"])
    CVTest = pm.MutableData("CVChickTest", taskResults["cvchickTask"])
    PCTBTest = pm.MutableData("PCTBTest", taskResults["pctbTask"])

    ## Margins

    goalSmallness = maxGoalSize - goalSize
    visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility, goalSmallness)))

    rightP = pm.Deterministic("rightPerformance", SimplePrMargin(rightAbility, goalRight))
    aheadP = pm.Deterministic("aheadPerformance", SimplePrMargin(aheadAbility, goalAhead))
    leftP = pm.Deterministic("leftPerformance", SimplePrMargin(leftAbility, goalLeft))

    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns, opTest)))
    flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns)))

    lavaP = pm.Deterministic("lavaP", SimplePrMargin(lavaAbility, lavaPresence))

    rampP = pm.Deterministic("rampP", SimplePrMargin(rampAbility, rampPresence))

    cvchickP = pm.Deterministic("cvChickP", SimplePrMargin(CVChickAbility, CVTest))
    pctbP = pm.Deterministic("pctbP", SimplePrMargin(PCTBAbility, PCTBTest))

    navP = pm.Deterministic("navP", (flatNavP * lavaP * rampP * rightP * aheadP * leftP * cvchickP * pctbP)) #, visualAcuityP))

    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, CVTest, PCTBTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    OPLOM = objPermanenceLOMargin(objPermAbility, opTest, cvchickP, pctbP, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)

    objPermP = pm.Deterministic("objPermP", logistic(OPLOM))

    noise = 1 - np.mean(results)  # With this noise is complementary to result prior.
    noisePar = pm.Uniform("noisePar", 0, 1)
    finalP = pm.Deterministic("finalP", (1-noisePar)*(objPermP * navP * visualAcuityP)+(noisePar*noise))

    #finalP = pm.Deterministic("finalP", (objPermP * navP * visualAcuityP))

    taskPerformance = pm.Bernoulli("taskPerformance", finalP, observed=results)

  pm.model_graph.model_to_graphviz(m)

  return m, abilityMin, abilityMax

In [ ]:
m, min, max = SetupModel(synthetic_agents_data, uniformAbilitySlack=1.4, agent_type='agent', agent_name="perfectAgent", sample = None)
gv = pm.model_graph.model_to_graphviz(m)
gv.format="png"
gv.render
#gv.render(directory='viz')
#Image("viz/Digraph.gv.png")
gv

In [ ]:
def SetupModelSimple(taskResultsAll, uniformAbilitySlack, agent_type, agent_name = None, sample = 500):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests.
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  if agent_type == "agent":
    taskResults = taskResultsAll.dropna(subset = [agent_name])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults[agent_name]
  elif agent_type == "child":
    taskResults = taskResultsAll.dropna(subset = ['success'])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults['success']
  else:
    print("Agent not recognised. Quitting.")
    return



  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistance = taskResults["cityBlockDistanceToGoal"].max()
  maxTurns = taskResults["minNumTurnsRequired"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max()
  maxPermAbility = ((taskResults["cityBlockDistanceToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxFlatNav = ((taskResults["cityBlockDistanceToGoal"]*taskResults["minNumTurnsRequired"])).max() *uniformAbilitySlack
  #maxPermAbility = maxChoices * uniformAbilitySlack
  #maxMemoryAbility = maxDistance * uniformAbilitySlack
  m = pm.Model()
  with m:

    objPermAbility = pm.Uniform("objPermAbility", 0, maxPermAbility)  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude
    abilityMin["objPermAbility"] = 0
    abilityMax["objPermAbility"] = maxPermAbility

    #memoryAbility = pm.Uniform("memoryAbility", 0, maxMemoryAbility)  # [0,maxMemoryAbility] This is the same as above, but we have the ability in the right magnitude
    #abilityMin["memoryAbility"] = 0
    #abilityMax["memoryAbility"] = maxMemoryAbility

    #Visual acuity
    #maxVisualAcuityAbility = maxGoalSize* uniformAbilitySlack
    #visualAcuityAbility = pm.Uniform("visualAcuityAbility", 0, maxVisualAcuityAbility)
    #abilityMin["visualAcuityAbility"] = 0
    #abilityMax["visualAcuityAbility"] = maxVisualAcuityAbility

    # Flat Navigation Ability
    #flatNavAbility = pm.Uniform("flatNavAbility", 0, maxDistance*maxTurns)      # how much navigation is involved, i.e, how far away and how circuitous is the path to the goal?
    #flatNavAbility = pm.Uniform("flatNavAbility", 0, maxTurns*maxDistance)
    #abilityMin["flatNavAbility"] = 0
    #abilityMax["flatNavAbility"] = maxDistance*maxTurns
    #abilityMax["flatNavAbility"] = maxFlatNav

    # Lava Ability
    #lavaAbility = pm.Beta("lavaAbility", 1,1)                        # [0,1] Specific ability with lava
    #abilityMin["lavaAbility"] = 0
    #abilityMax["lavaAbility"] = 1

    # Ramp Ability
    #rampAbility = pm.Beta("rampAbility",1,1)                          # [0,1] Specific ability with ramps
    #abilityMin["rampAbility"] = 0
    #abilityMax["rampAbility"] = 1

    # Goal Right Ability
    #rightAbility = pm.Beta("rightAbility", 1, 1)
    #abilityMin["rightAbility"] = 0
    #abilityMax["rightAbility"] = 1

    # Goal Left Ability
    #leftAbility = pm.Beta("leftAbility", 1, 1)
    #abilityMin["leftAbility"] = 0
    #abilityMax["leftAbility"] = 1

    # Goal Ahead Ability
    #aheadAbility = pm.Beta("aheadAbility", 1, 1)
    #abilityMin["aheadAbility"] = 0
    #abilityMax["aheadAbility"] = 1

    # CV Chick Ability
    #CVChickAbility = pm.Beta("CVChickAbility", 1, 1)
    #abilityMin["CVChickAbility"] = 0
    #abilityMax["CVChickAbility"] = 1

    # PCTB Ability
    #PCTBAbility = pm.Beta("PCTBAbility", 1, 1)
    #abilityMin["PCTBAbility"] = 0
    #abilityMax["PCTBAbility"] = 1



    ## Environment variables as Deterministic (about the instance)

    #lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values)
    #rampPresence = pm.MutableData("rampPresence", taskResults["taskCriticalRampPresence"].values)
    #lightsOutPresence = pm.MutableData("lightsOutPresence", taskResults["lightsOutPresence"].values)
    #numGoals = pm.MutableData("numberOfGoals", taskResults["numGoalsAll"].values)
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values)
    #goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values)
    goalDist = pm.MutableData("goalDistance", taskResults["cityBlockDistanceToGoal"])
    #numTurns = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsRequired"])
    #goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"])
    #goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"])
    #goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"])
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"])
    #CVTest = pm.MutableData("CVChickTest", taskResults["cvchickTask"])
    #PCTBTest = pm.MutableData("PCTBTest", taskResults["pctbTask"])

    ## Margins

    #goalSmallness = maxGoalSize - goalSize
    #visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility, goalSmallness)))

    #rightP = pm.Deterministic("rightPerformance", SimplePrMargin(rightAbility, goalRight))
    #aheadP = pm.Deterministic("aheadPerformance", SimplePrMargin(aheadAbility, goalAhead))
    #leftP = pm.Deterministic("leftPerformance", SimplePrMargin(leftAbility, goalLeft))

    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns, opTest)))
    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavMargin(flatNavAbility, goalDist, numTurns)))
    #flatNavP = pm.Deterministic("flatNavP", logistic(flatNavAbility - (numTurns * goalDist)))
    #lavaP = pm.Deterministic("lavaP", SimplePrMargin(lavaAbility, lavaPresence))

    #rampP = pm.Deterministic("rampP", SimplePrMargin(rampAbility, rampPresence))

    #cvchickP = pm.Deterministic("cvChickP", SimplePrMargin(CVChickAbility, CVTest))
    #pctbP = pm.Deterministic("pctbP", SimplePrMargin(PCTBAbility, PCTBTest))

    #navP = pm.Deterministic("navP", (flatNavP * lavaP * rampP * rightP * aheadP * leftP * cvchickP * pctbP)) #, visualAcuityP))

    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, CVTest, PCTBTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)
    #OPLOM = objPermanenceLOMargin(objPermAbility, opTest, cvchickP, pctbP, lightsOutPresence, goalDist, numChoices, maxPermAbility, lightsOutPenalisation=(uniformAbilitySlack-0.35), uniformAbilitySlack=uniformAbilitySlack)

    #OPPerformance = (objPermAbility  - ((goalDist * numChoices)*opTest)) #this returns a value that is lower when (a) ability is lower, (b) the goal is occluded for longer or there are more positions where it could be occluded, or (c) when the lights go out (by a small penalisation)
    #OPPerformance = (objPermAbility  - ((goalDist * numChoices)))
    #OPPerformance = (objPermAbility  - numChoices)
    OPPerformance = (objPermAbility  - ((goalDist*opTest) * (numChoices*opTest)))
    #OPability = maxPermAbility - ((maxPermAbility-OPPerformance)*opTest)
    #objPermP = pm.Deterministic("objPermP", logistic(OPLOM))
    objPermP = pm.Deterministic("objPermP", logistic(OPPerformance))


    #flatNavP = pm.Deterministic("flatNavP", logistic((flatNavAbility - (numTurns * (goalDist * (1-opTest))))* (opTest*objPermP)))
    #noise = 1 - np.mean(results)  # With this noise is complementary to result prior.
    #noisePar = pm.Uniform("noisePar", 0, 1)
    #finalP = pm.Deterministic("finalP", (1-noisePar)*(objPermP * navP * visualAcuityP)+(noisePar*noise))

    #finalP = pm.Deterministic("finalP", (objPermP * flatNavP))
    #finalP = pm.Deterministic("finalP", (1-((1-objPermP)*(1-flatNavP)))) #compensatory finalPerformance

    #taskPerformance = pm.Bernoulli("taskPerformance", flatNavP, observed=results)
    #taskPerformance = pm.Bernoulli("taskPerformance", finalP, observed=results)
    taskPerformance = pm.Bernoulli("taskPerformance", objPermP, observed=results)
  pm.model_graph.model_to_graphviz(m)

  return m, abilityMin, abilityMax

In [ ]:
m, min, max = SetupModelSimple(synthetic_agents_data, uniformAbilitySlack=1.4, agent_type='agent', agent_name="perfectAgent", sample = None)
gv = pm.model_graph.model_to_graphviz(m)
gv.format="png"
gv.render
#gv.render(directory='viz')
#Image("viz/Digraph.gv.png")
gv

In [ ]:
# A function for pulling out the means and standard deviations for abilities of interest
def analyzeAgentResults(agentName, agentData, abilitiesToShow):

  abilityMeans = [] # empty list to add in ability means to
  abilitySDs = []
  for a in abilitiesToShow: #iterate through each ability, add posterior mean to dataframe, and plot posterior

    posteriorMean = float(np.mean(agentData['posterior'][a])) # calculate posterior mean
    posteriorSD = float(np.std(agentData['posterior'][a])) #calculate posterior sd
    abilityMeans.append(posteriorMean)
    abilitySDs.append(posteriorSD)

  return abilityMeans, abilitySDs

# A function for padding the testing data with 0s for prediction
def pad(testingData, trainingDataSize):
    return testingData.append(pd.Series(np.zeros(trainingDataSize-len(testingData), dtype=int)))

# A function for making predictions based on a fitted measurement layout
def predict(m, agentData, dfTest, agent, len_training):
  with m:
    # set the data for prediction
    pm.set_data({"lavaPresence": pad(dfTest["lavaPresence"], len_training)})
    pm.set_data({"lightsOutPresence": pad(dfTest["lightsOutPresence"], len_training)})
    pm.set_data({"rampPresence": pad(dfTest["taskCriticalRampPresence"], len_training)})
    pm.set_data({"numChoices": pad(dfTest["numChoices"], len_training)})
    pm.set_data({"minTurnsToGoal": pad(dfTest["minNumTurnsRequired"], len_training)})
    pm.set_data({"goalDistance": pad(dfTest["cityBlockDistanceToGoal"], len_training)})
    pm.set_data({"goalSize": pad(dfTest["mainGoalSize"], len_training)})
    pm.set_data({"goalRight": pad(dfTest["goalRightRelToStart"], len_training)})
    pm.set_data({"goalAhead": pad(dfTest["goalCentreRelToStart"], len_training)})
    pm.set_data({"goalLeft": pad(dfTest["goalLeftRelToStart"], len_training)})
    pm.set_data({"CVChickTest": pad(dfTest["cvchickTask"], len_training)})
    pm.set_data({"PCTBTest": pad(dfTest["pctbTask"], len_training)})
    pm.set_data({"allocentricOPTest": pad(dfTest["goalBecomesAllocentricallyOccluded"], len_training)})

    predictions=pm.sample_posterior_predictive(agentData, var_names=["finalP"], return_inferencedata=False,predictions=True,extend_inferencedata=False)
    predictionChainRuns =predictions["finalP"][:,:,0:len(dfTest)]
    predictionsInstance = np.mean(predictionChainRuns, (0,1))

    return predictionsInstance,  dfTest[agent].to_numpy()


def brierScore(preds, outs):
    return 1/len(preds) * sum( (preds-outs)**2 )

def brierDecomp(preds, outs):

  brier= 1/len(preds) * sum( (preds-outs)**2 )
  ## bin predictions
  bins = np.linspace(0,1,11)
  binCenters = (bins[:-1] +bins[1:]) /2
  binPredInds = np.digitize(preds,binCenters)
  binnedPreds = bins[binPredInds]

  binTrueFreqs = np.zeros(10)
  binPredFreqs = np.zeros(10)
  binCounts = np.zeros(10)

  for i in range(10):
      idx = (preds >= bins[i]) & (preds < bins[i+1])

      binTrueFreqs[i] = np.sum(outs[idx])/np.sum(idx) if np.sum(idx) > 0 else 0
     # print(np.sum(outs[idx]), np.sum(idx), binTrueFreqs[i])
      binPredFreqs[i] = np.mean(preds[idx]) if np.sum(idx) > 0 else 0
      binCounts[i] = np.sum(idx)

  calibration = np.sum(binCounts * (binTrueFreqs - binPredFreqs) ** 2) / np.sum(binCounts) if np.sum(binCounts)>0 else 0
  refinement = np.sum(binCounts * (binTrueFreqs *(1 - binTrueFreqs))) / np.sum(binCounts) if np.sum(binCounts)> 0 else 0
  # Compute refinement component
  #refinement = brier - calibration
  return brier, calibration, refinement



In [ ]:
agents_training, agents_test = train_test_split(agent_data, test_size=0.2)
children_training, children_test = train_test_split(children_data, test_size=0.2)


In [ ]:
m, min, max = SetupModel(children_training, uniformAbilitySlack=1.4, agent_type='child', sample = None)

In [ ]:
with m:
    childData = pm.sample(500, target_accept=0.95) #Might need more samples to converge

In [ ]:
abilitiesToShow = ["objPermAbility", "visualAcuityAbility", "flatNavAbility", "lavaAbility", "rampAbility", "rightAbility", "leftAbility", "aheadAbility", "CVChickAbility", "PCTBAbility", "noisePar"]

m, min, max = SetupModel(children_test, uniformAbilitySlack=1.4, agent_type='child', sample = None)

mu, sd  = analyzeAgentResults("children", childData, abilitiesToShow)
print(mu)
print(sd)
#predictions, testOutputs = predict(m, childData, children_test, "success", len(children_training["success"]))

In [ ]:
m, min, max = SetupModelSimple(synthetic_agents_data, uniformAbilitySlack=1.2, agent_type='agent', agent_name="perfectAgent", sample = None)

In [ ]:
with m:
    agentData = pm.sample(2000, target_accept=0.95) #Might need more samples to converge

In [ ]:
#abilitiesToShow = ["objPermAbility", "flatNavAbility"]
abilitiesToShow = ["objPermAbility"]
mu, sd  = analyzeAgentResults("perfectAgent", agentData, abilitiesToShow)
mean_success = np.mean(synthetic_agents_data['perfectAgent'])
print(mean_success)
print(mu)
print(sd)
print(max)

In [ ]:
agentData["posterior"]["objPermPDet"]

In [ ]:
fig, (ax1) = plt.subplots(1,1)
az.plot_posterior(agentData["posterior"]["objPermAbility"], ax=ax1);
plt.show()

In [ ]:
abilitiesToShow = ["objPermAbility", "visualAcuityAbility", "flatNavAbility", "lavaAbility", "rampAbility", "rightAbility", "leftAbility", "aheadAbility", "CVChickAbility", "PCTBAbility", "noisePar"]
mu, sd  = analyzeAgentResults("perfectAgent", agentData, abilitiesToShow)
print(mu)
print(sd)
print(max)

In [ ]:
agentName = "Vanilla_Braitenberg_15_rays_over_60_degs_3761"
abilitiesToShow = ["objPermAbility", "visualAcuityAbility", "flatNavAbility", "lavaAbility", "rampAbility", "rightAbility", "leftAbility", "aheadAbility", "CVChickAbility", "PCTBAbility", "noisePar"]

m, min, max = SetupModel(agents_test, uniformAbilitySlack=1.4, agent_type='agent', agent_name=agentName)

mu, sd  = analyzeAgentResults(agentName, agentData, abilitiesToShow)
print(mu)
print(sd)
#predictions, testOutputs = predict(m, agentData, agents_test, agentName, len(agents_training[agentName]))

In [ ]:

abilityDF = pd.DataFrame( {'Ability': abilitiesIncludingSuccess})
meansDF = pd.DataFrame({"Ability": abilitiesIncludingSuccess})
stdDevDF = pd.DataFrame({"Ability": abilitiesIncludingSuccess})
modelBrierScores = []
aggregateBrierScores = []
modelCalibrations = []
aggregateCalibrations = []
modelRefinements = []
aggregateRefinements = []

agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(agent_data[agentName]), len(testOutputs)), testOutputs)
aggregateBrierScores.append(agentAggBrierScore)
aggregateCalibrations.append(agentAggCalibration)
aggregateRefinements.append(agentAggRefinement)
modelBrierScores.append(agentBrierScore)
modelCalibrations.append(agentCalibration)
modelRefinements.append(agentRefinement)
abilityMeans= [str(round(mu_i, 2))+" " for mu_i in mu]  # add ability mean estimate to list for this agent
abilitySDs = [""+str(round(sd_i,2)) for sd_i in sd ]

In [ ]:
agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
abilityMeans= [str(round(mu_i, 2))+" " for mu_i in mu]  # add ability mean estimate to list for this agent
abilitySDs = [""+str(round(sd_i,2)) for sd_i in sd ]
print(agentBrierScore)
print(agentCalibration)
print(agentRefinement)
print(abilityMeans)
print(abilitySDs)

In [ ]:
brier= 1/len(predictions) * sum( (predictions-testOutputs)**2 )
print(sum( (predictions-testOutputs)**2))
# ## bin predictions
# bins = np.linspace(0,1,11)
# binCenters = (bins[:-1] +bins[1:]) /2
# binPredInds = np.digitize(preds,binCenters)
# binnedPreds = bins[binPredInds]
# binTrueFreqs = np.zeros(10)
# binPredFreqs = np.zeros(10)
# binCounts = np.zeros(10)
# for i in range(10):
#   idx = (preds >= bins[i]) & (preds < bins[i+1])

#   binTrueFreqs[i] = np.sum(outs[idx])/np.sum(idx) if np.sum(idx) > 0 else 0
#   # print(np.sum(outs[idx]), np.sum(idx), binTrueFreqs[i])
#   binPredFreqs[i] = np.mean(preds[idx]) if np.sum(idx) > 0 else 0
#   binCounts[i] = np.sum(idx)

# calibration = np.sum(binCounts * (binTrueFreqs - binPredFreqs) ** 2) / np.sum(binCounts) if np.sum(binCounts)>0 else 0
# refinement = np.sum(binCounts * (binTrueFreqs *(1 - binTrueFreqs))) / np.sum(binCounts) if np.sum(binCounts)> 0 else 0
# # Compute refinement component
# #refinement = brier - calibration



In [ ]:
brierDF = pd.DataFrame({"Model Brier Score":modelBrierScores, "Aggregate Brier Scores": aggregateBrierScores, "Model Calibration": modelCalibrations, "Aggregate Calibration":aggregateCalibrations, "Model Refinement":modelRefinements, "Aggregate Refinement":aggregateRefinements,  "Model Better? (Based on Brier Score)":np.array(modelBrierScores)<np.array(aggregateBrierScores), "Success Score":meansDF.iloc[9][1:]})

In [ ]:
taskResults = agent_data[agentName] #Grab the column of results for that agent
print(agent_data)
print(np.mean(taskResults)) #print mean performance for the agent across all tasks
m = SetupModel(taskResults) #Define the model using the setupModel function. Needs to be redefined each run after taskResults are updated or PyMC won't use the latest taskResults

mu, sd  = analyzeAgentResults(agentName, agentData)
predictions, testOutputs = predict(m, agentData, agents_test[agentName], len(agents_test[agentName]))
agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(taskResults), len(testOutputs)), testOutputs)
aggregateBrierScores.append(agentAggBrierScore)
aggregateCalibrations.append(agentAggCalibration)
aggregateRefinements.append(agentAggRefinement)
modelBrierScores.append(agentBrierScore)
modelCalibrations.append(agentCalibration)
modelRefinements.append(agentRefinement)
abilityMeans= [str(round(mu_i, 2))+" " for mu_i in mu]  # add ability mean estimate to list for this agent
abilitySDs = [""+str(round(sd_i,2)) for sd_i in sd ]

In [ ]:
mu

In [ ]:
sd

In [ ]:
max

In [ ]:
##### Sample results for each agent using the model defined in the function above.
allAgentData = [] #blank list for entering the outputs of the pm sampling for each agent
agentNames = [] #blank list for the agent names
agentColList = agentCols.tolist()
print(agentColList)
numAgents = len(agentColList) #Number of agents, replace with a small number for testing to save time
print(numAgents)
#numAgents=3
abilitiesToShow = ["objPermAbility", "flatNavAbility", "visualAcuityAbility","lavaAbility", "platformAbility", "rampAbility", "memoryAbility", "rightLeftBias", "noisePar"]

abilitiesIncludingSuccess = abilitiesToShow + ["Success"]
abilityDF = pd.DataFrame( {'Ability': abilitiesIncludingSuccess})
meansDF = pd.DataFrame({"Ability": abilitiesIncludingSuccess})
stdDevDF = pd.DataFrame({"Ability": abilitiesIncludingSuccess})
modelBrierScores = []
aggregateBrierScores = []
modelCalibrations = []
aggregateCalibrations = []
modelRefinements = []
aggregateRefinements = []


for i in range(numAgents): # Iterate through each agent.
  agent = agentCols[i] #Get the agent name
  taskResults = df[agent] #Grab the column of results for that agent
  print(agent)
  print(np.mean(taskResults)) #print mean performance for the agent across all tasks
  m = SetupModel(taskResults) #Define the model using the setupModel function. Needs to be redefined each run after taskResults are updated or PyMC won't use the latest taskResults

  #Now sample based on this agent's performance
  with m:
    agentData = pm.sample(1000, target_accept=0.95) #Might need more samples to converge

  mu, sd  = analyzeAgentResults(agent, agentData)
  predictions, testOutputs = predict(m, agentData, dfTest, agent)
  agentBrierScore, agentCalibration, agentRefinement = brierDecomp(predictions, testOutputs)
  agentAggBrierScore, agentAggCalibration, agentAggRefinement = brierDecomp(np.repeat(np.mean(taskResults), len(testOutputs)), testOutputs)
  aggregateBrierScores.append(agentAggBrierScore)
  aggregateCalibrations.append(agentAggCalibration)
  aggregateRefinements.append(agentAggRefinement)
  modelBrierScores.append(agentBrierScore)
  modelCalibrations.append(agentCalibration)
  modelRefinements.append(agentRefinement)
  abilityMeans= [str(round(mu_i, 2))+" " for mu_i in mu]  # add ability mean estimate to list for this agent
  abilitySDs = [""+str(round(sd_i,2)) for sd_i in sd ]


  abilityMeans = abilityMeans + [str(round(np.mean(taskResults),2))+" "]
  abilitySDs = abilitySDs + [str(round(np.std(taskResults), 2))]
  abilityDF[agent] = list(zip(abilityMeans, abilitySDs))
  mu=mu+[np.mean(taskResults)]
  sd=sd+[np.std(taskResults)]
  meansDF[agent]=[round(mu_i,2) for mu_i in mu]
  stdDevDF[agent]=[round(sd_i,2) for sd_i in sd]